1. The Docker container is the environment for running your application. It's a complete, isolated operating system with its own Python installation and dependencies. No need for a traiditonal virtual env.
2.A local venv is a lightweight tool created for your code editor to understand your project's dependencies.


user model 

user can be created by admin only, so from django admin panel?
User log in only, no registration. 

django admin and CRM admin are different users.   django admin - project's supervisor, crm admin is the owner of the crm. 


Now---

create User model
makemigrations -- migrate


next - serializer or too soon ?


what was done:
1. virtual env, start django project, start users app
2. Added custom User model, email for login instead of username; ROLE,VERSION
3. Added rest framework and jwt to installed APPS, settings.py -> JWT for default auth
4. Added custom user manager, self.create_user(email, password, **extra_fields)
5. postgress DB settings, env settings -> secret and postgress host, pass, etc
6. Backend dockerfile, and docker compose yml - services : postgress and backend


# 26/07/25

Serializers - the fields we want to expose in our API, so not neccessarily 1-1 with the model's fields, e.g. 
we don't want to send password over the API.
The ModelSerializer inspects your User model and generates these fields for you automatically

1. user - model instance
2. serializer  = UserSerializer(user)    -> translate model instance to Python native data types
serializer.data -> {'id': 2, 'first_name': 'bob', 'last_name': 'diku', 'email': 'dakux@yahoo.com', 'version': 1, 'is_active': True, 'role': 'USER'}

3. Lastly render the data to JSON --> 
content = JSONRenderer().render(serializer.data)
content
b'{"id":2,"first_name":"bob","last_name":"diku","email":"dakux@yahoo.com","version":1,"is_active":true,"role":"USER"}'


---- Deserialization ---  From JSON to stream to native python datatypes
1. Parse a stream into Python native datatypes
>>> import io
>>> stream  = io.BytesIO(content)
>>> data = JSONParser().parse(stream)

2. then we restore those native datatypes into a fully populated object instance.
serializer = SnippetSerializer(data=data)
serializer.is_valid()
# True
serializer.validated_data
# {'title': '', 'code': 'print("hello, world")', 'linenos': False, 'language': 'python', 'style': 'friendly'}
serializer.save()
# <Snippet: Snippet object>

The ModelSerializer's default .create() method knows how to create a new User model instance. It effectively runs:
new_user = User.objects.create(**validated_data)


---  Views ----
Need to write login auth view 
Map the view to url

django does the auth itself? 


Then login wireframe / frontend -> dashboard

users view - accessable by the admin USER only. can manage users. 





